In [127]:
import os

import pandas as pd
import pandera as pa

Welcome to this mini-tutorial on the python library pandera. Pandera lets you validate dataframe objects against specified data schemas. This can be easily integrated into your data pipelines in order to continuously test your data, spot data errors or schema updates, and generally support data quality monitoring.

Let us start by reading in a small dataframe. Its column names should already give you a hint on each intended data type.

In [125]:
df = pd.read_csv(os.path.join("dummy_data", "dummy.csv"))

In [ ]:
df

However, to raise any doubts, here is the schema again:

| Column/variable name | Data type                                     |
|----------------------|-----------------------------------------------|
| var_bool             | boolean                                       |
| var_signed_int       | signed integer (positive and negative values) |
| var_unsigned_int     | unsigned integer (only non-negative values)   |
| var_float            | floating point number                         |
| var_str              | string                                        |
| var_date             | date (format YYYY-MM-DD)                      |
| var_categorical      | categorical                                   |
| var_constant         | constant                                      |


## Define pandera schema as DataFrameSchema object

Let's try to write a schema for it! For this we have to create a pandera DataFrameSchema object:

In [ ]:
schema = pa.DataFrameSchema()
print(type(schema))

Now we add a first column (a pandera Column object):

In [132]:
schema = schema.add_columns({"Var_bool": pa.Column(bool)})

Now we can already try to validate our dataframe against the specified schema. The validate() function throws an error if one or multiple schema errors are encountered. Otherwise it just returns the validated dataframe.

In [ ]:
schema.validate(df)

Ooops, the validate function tells us that the specified column in the schema "Var_bool" is no in the dataframe. That's because the column in the dataframe is written in lowercase "var_bool". Let us try that again then. We try to rename the column.

In [134]:
# TODO: correct the name of the column in the schema. The rename_columns function expects a dictionary with old names as keys and new names as values (replace the dots with the dictionary)
schema = schema.rename_columns(...)

Let's look at the updated schema and try validation again.

In [ ]:
schema.validate(df)

The dataframe was returned and survived our first validation. Given we specified the column var_bool with a correct type, the validate function does not throw an error. However, what if we want it to throw an error because there are variables in the dataframe which are not mentioned in the schema (yet)? Let us look at the schema.

In [ ]:
print(schema)

Apart from the columns, there are a couple other properties of interest here.

**checks**: Here you can add user-defined checks for columns or groups of columns. Checks can also be directly given as an argument to a column object, but are limited to that column then. We will look into that later.

**coerce**: If true pandera tries to coerce/cast the column into the specified type. This is helpful if your columns in the dataframe are not specifically typed, but you just read it the frame from, e.g., a csv file. For instance, the inferred data type for a column of zeros and ones by pandas would probably be integer. However, if you specify its datatime to be bool in the schema and set coerce to True, validation should not throw an error because coercion should work.

**strict**: If set to true means that if we want additional unexpected columns in the dataframe to throw a validation error, we have to set the attribute "strict" to True.

**ordered**: If set to true, the columns in the dataframe must appear in the same order as the columns in the schema.

If you want to know more about all configurable properties of a DataFrameSchema, have a look at the [docs](https://pandera.readthedocs.io/en/stable/reference/generated/pandera.api.pandas.container.DataFrameSchema.html#pandera.api.pandas.container.DataFrameSchema).

In [ ]:
# TODO: Update a schema property such that we will receive a validation error because there are additional columns in our original dataframe that are not (yet) in the schema.
schema

In [ ]:
schema.validate(df)

Now we should get a schema error. The column from the dataframe "var_signed_int" does not appear in our schema. Let us redefine our schema and add all columns. We deliberately define a wrong datatype for all columns but the var_bool column here because we want to learn about lazy validation.

In [157]:
schema = pa.DataFrameSchema({"var_bool": pa.Column(bool),
                             "var_signed_int": pa.Column(bool),
                             "var_unsigned_int": pa.Column(bool),
                             "var_float": pa.Column(bool),
                             "var_str": pa.Column(bool),
                             "var_date": pa.Column(bool),
                             "var_categorical": pa.Column(bool),
                             "var_constant": pa.Column(bool)})

You might want to repeatedly execute validation to find out the errors. By default pandera will stop validation at the first encountered error. If you want to be informed about all errors during validation, you need to provide the lazy keyword to the validate function. This is useful when data validation is applied to an incrementing dataset in a pipeline. You want to be informed about all errors and not re-execute the pipeline again and again to detect all problems.

In [ ]:
schema.validate(df, lazy=True)

We see that there are 8 errors here. Let us try to fix them!

In [ ]:
# TODO: Update the schema below with the correct datatypes. Ignore the difference between unsigned and signed int for now
schema = pa.DataFrameSchema({"var_bool": pa.Column(bool),
                             "var_signed_int": pa.Column(...),
                             "var_unsigned_int": pa.Column(...),
                             "var_float": pa.Column(...),
                             "var_str": pa.Column(...),
                             "var_date": pa.Column(...),
                             "var_categorical": pa.Column(...),
                             "var_constant": pa.Column(...)})

In [ ]:
df.dtypes

**Hints:**
- Overall, [this list](https://pandera.readthedocs.io/en/stable/reference/dtypes.html) of data types might help, but for basic data types like integer you can also use pure Python built-in data types.
- Check out the pandera datatypes [pa.Category](https://pandera.readthedocs.io/en/stable/reference/generated/pandera.dtypes.Category.html) and [pa.DateTime](https://pandera.readthedocs.io/en/stable/reference/generated/pandera.dtypes.DateTime.html).
- If pandera complains about missing values in the dataframe during validation which are actually possible, you might want to consider changing the keyword "nullable" which by default is set to False.
- Like explained above coercion sometimes helps to convince pandera of the datatype you mean. If you check df.dtypes you will notice that most columns have the pandas-inferred datatype "object". Pandera accepts string for basically everything, but if you want to define a more specific datatype, try to set the coerce argument to True. You can also set it to true for the whole schema. This will overwrite the column-specific specification.

In [ ]:
schema.validate(df, lazy=True)

Now let us differentiate between the data types for the signed and unsigned integer. A possibility to do this is by working with [pandera Checks](https://pandera.readthedocs.io/en/stable/reference/generated/pandera.api.checks.Check.html).

In [ ]:
# TODO: Screen the possible checks and choose an appropriate one. Update the schema accordingly:
schema = schema.update_columns({"var_unsigned_int": {"dtype": int, "checks": [...]}})

Let us verify schema validation still runs through

In [ ]:
schema.validate(df, lazy=True)

Congrats! You wrote your first schema including a check.

## Schema inference

Imagine you have a complex dataset and you do not want to write the schema from scratch. After all, time is limited and manual processes are error-prone. Luckily, pandera comes with a function that can generate a first draft of a pandera schema. Let's try out which data types pandera infers from our dummy csv file.

In [ ]:
inferred_schema = pa.infer_schema(df)
print(inferred_schema)

As can be seen, the inference of bool, integer and floating point variables worked nicely. However, the data types of string variables, categorical variables and date variables have to be further specified as they are currently just typed "objects".

⚠️ The results from the infer_schema function from pandera should always be seen as a first draft and should be updated using your domain knowledge.

You can edit the schema directly (e.g. using the update_columns() function). What I prefer is to save the schema as a file and edit it there. Let's try that. Pandera allows you to save the schemas as yaml-file, json-file or python-script.


## Save schema as file

In [189]:
inferred_schema.to_json(os.path.join("dummy_data", "inferred_dummy_schema.json"), indent=2)
inferred_schema.to_script(os.path.join("dummy_data", "inferred_dummy_schema.py"))
# TODO: open the schema (choose one of the two files and edit it)

When you are done editing, you can read the schema again and validate your dataframe against it. From the python script you can directly import the schema.

In [184]:
schema = pa.DataFrameSchema.from_json(os.path.join("dummy_data", "inferred_dummy_schema.json"))
# OR:
# from inferred_dummy_schema import schema
# schema = schema

schema.validate(df)

Saving the schema as a file is useful if you want to include data validation in your data pipeline. It also makes versioning of your data schema easier.